In [1]:
using LowLevelFEM, LinearAlgebra

In [2]:
gmsh.initialize()
gmsh.open("cantilever.geo")

Info    : Reading 'cantilever.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.000608386s, CPU 0.000536s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.000392313s, CPU 0.000392s)
Info    : Meshi

In [3]:
mat = material("body")
prob = Problem([mat])

Problem("cantilever", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0e-7, 0.1, 1.0)], 1.0, 1836, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [4]:
R = 201
x(X, Y, Z) = X
y(X, Y, Z) = (Y + R) * cos(Z / R) - R
z(X, Y, Z) = (Y + R) * sin(Z / R)

r = VectorField(prob, "body", [x, y, z]);

In [5]:
R = 200
#x(X, Y, Z) = X
#y(X, Y, Z) = (Y + R) * cos(Z / R) - R
#z(X, Y, Z) = (Y + R) * sin(Z / R)

r2 = VectorField(prob, "body", [x, y, z]);

In [6]:
showDeformationResults(r, name="r(t)")
showDeformationResults(r2, name="r(t+Δt)")

2

In [7]:
dr = r2 - r
u = dr;
v = dr;

In [8]:
showDoFResults(v, name="v(t)")

3

In [9]:
F = r ∘ ∇
F2 = r2 ∘ ∇
dF = v ∘ ∇
l = dF * inv(F);

In [10]:
d = (l + l') / 2;
w = (l - l') / 2;

In [11]:
rnd1 = rand(3)
v1 = VectorField(prob, "body", rnd1 / norm(rnd1))
rnd2 = rand(3)
v2 = VectorField(prob, "body", rnd2 / norm(rnd2))
v2 = v1 × v2 / norm(v1 × v2)
v3 = v1 × v2

ex = VectorField(prob, "body", [1, 0, 0])
ey = VectorField(prob, "body", [0, 1, 0])
ez = VectorField(prob, "body", [0, 0, 1])

Q = v1 ∘ ex + v2 ∘ ey + v3 ∘ ez;

I = TensorField(prob, "body", [1 0 0; 0 1 0; 0 0 1]);

In [12]:
E = (F' * F - I) / 2
E2 = (F2' * F2 - I) / 2
e = inv(F') * E * inv(F)
e2 = inv(F2') * E2 * inv(F2)
de = e2 - e

dE = E2 - E
d = inv(F') * dE * inv(F);

In [13]:
showDoFResults(E, name="E")
showDoFResults(e, name="e")
showDoFResults(dE, name="dE/dt")
showDoFResults(de, name="de/dt")
showDoFResults(d, name="d")
#showDoFResults(de - d, name="de/dt - d")

8

In [14]:
showDoFResults(w * u, name="w.u")
showDoFResults(v - w * u, name="v-w.u")

showDoFResults(de - w * e + e * w, name="de-w.e+e.w")

11

In [15]:
D = F - I
ε = (D + D') / 2;

In [16]:
showDoFResults(ε, name="ε")

12

In [17]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.15.0-git
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib tinyobjloader
FLTK version  : 1.3.8
OCC version   : 7.9.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
Fontconfig warning: using without calling FcInit()


In [18]:
gmsh.finalize()